In [ ]:
!git clone "https://github.com/s-ankur/wikiextract"
%cd wikiextract/

Run the setup file. It downloads the required repos and data dumps


In [ ]:
!bash setup.sh 

You need to run these two steps after every failed run. Because we use these to track the progress.

In [ ]:
!rm hindi.output hindi-pos-tagger-3.0/hindi.input.txt

In [ ]:
rm extracted

This step is used to run the actual code. it might take some time to run.

In [ ]:
!bash run_hiwiki.sh

+ SEED=42
+ OUTPUT_SENT=1000000
+ HIWIKI_FILE=hiwiki-pages-meta-current.xml
+ EXTRACTED_FILE=extracted
+ EXTRACT_TEXT='python3 WikiExtractor.py --quiet --no_templates --filter_disambig_pages   --min_text_length 20   $HIWIKI_FILE -b 5G -o -'
+ TOKENIZE_SENTENCE='python3 indic_sentence_tokenizer.py'
+ SELECT_RANDOM_SENTENCE='shuf -n $OUTPUT_SENT --random-source=<(get_seeded_random $SEED)'
+ INSERT_ERRORS='python3 insert_errors.py hindi.output  --single --edits'
+ '[' '!' -f extracted ']'
+ eval 'python3 WikiExtractor.py --quiet --no_templates --filter_disambig_pages   --min_text_length 20   $HIWIKI_FILE -b 5G -o ->extracted'
++ python3 WikiExtractor.py --quiet --no_templates --filter_disambig_pages --min_text_length 20 hiwiki-pages-meta-current.xml -b 5G -o -
+ '[' -f extracted ']'
+ '[' '!' -f hindi-pos-tagger-3.0/hindi.input.txt ']'
+ wc -l extracted
1419317 extracted
+ eval 'python3 indic_sentence_tokenizer.py<extracted>tmp.tok'
++ python3 indic_sentence_tokenizer.py
+ eval shuf -n '$

A sample of the generated sentences

In [ ]:
!head -4000 hiwiki.augmented.edits|python scripts/convert_to_wdiff.py|shuf -n 40

यह नदी काठगोदाम से होकर भी [-गुजरते-] {+गुजरती+} है , जिसके किनारे कई शानदार प्राकृतिक आकर्षण मौजूद हैं .
यह पोकीमॉन वीडियो गेम सीरीज़ पर आधारित है और पोकीमॉन फ्रेंचाइज़ी [-के-] {+का+} हिस्सा है .
देश के विभिन्न हिस्सों से जैन समाज के लोग यहां बड़ी आस्था [-का-] {+के+} साथ मूर्तियों के दर्शन करने आते हैं .
ओरछा के साथ टीकमगढ़ का राज्येतिहास भी यही [-हैं-] {+है+} .
27 दिसम्बर 1975 को भारत के इतिहास [-का-] {+के+} सबसे बडी़ खान दुर्घटना धनबाद से 20 किलोमीटर दूर चासनाला में घटी , .
यह संगठन निरंतर प्रगति की राह पर कार्य [-करता-] {+करते+} हुए दक्षिण पूर्वी क्षेत्र में आर्थिक विकास के लिए सहयोग करने की दिशा में कार्यरत है .
सभी नॉकआउट चरण मैचों को पुरुषों के टूर्नामेंट से प्रासंगिक मैच के साथ डबल हेडर के रूप में [-खेले-] {+खेला+} जाएगा .
डोगो द्वीप [-के-] {+का+} प्रमुख नगर सैगो है जो शिमाने द्वीप के सकाई बंदरगाह से ४० मील दूर है .
2011 की जनगणना के अनुसार विशाखापत्तनम शहर की जनसंख्या 1 , 728 , 128 [-हैं-] {+है+} .
पिंडी भिजवाया और खुद कराची के रास्ते पाकिस्तान [-पहुंचा-] {+पहुंचे+} तो पता चला

This is the output file

In [ ]:
!wc hiwiki.augmented.edits

   7778655   91007242 1112449366 hiwiki.augmented.edits


In [ ]:
total=7778655;
if total%3 :raise Exception("check the augmented file")
total_edits=total//3
train=int(total_edits*.8)*3
val = total - train 
tmp3=train+val
print(train,val)

6222924 1555731


In [ ]:
!head -$tmp3 hiwiki.augmented.edits>train
!tail -$val hiwiki.augmented.edits>val
!mkdir -p data

In [ ]:
def convert_to_srctrg(inp, src, trg):
    bad_words = ('### comment:', '### contributor:', '### id:', '### page:', '### timestamp:', '###')
    i = 0
    for line in inp:
        if line.isspace():
            continue
        if line.startswith(bad_words, 0):
            continue
        if i % 2 == 0:
            src.write(line)
        else:
            trg.write(line)
        i = i + 1


convert_to_srctrg(open('train'), open('data/train_merge.src','w'), open('data/train_merge.tgt','w'))
convert_to_srctrg(open('val'), open('data/valid.src','w'), open('data/valid.tgt','w'))

In [ ]:
!rm train val
!bash shuffle.sh data/train_merge 42
!bash shuffle.sh data/valid 42

In [ ]:
!wc data/*

   2592885   45503621  555239633 data/train_merge.src
   2592885   45503621  554616848 data/train_merge.tgt
    518577    9100826  111048886 data/valid.src
    518577    9100826  110925847 data/valid.tgt
   6222924  109208894 1331831214 total


In [ ]:
mv data ../data